# Canonical MPS forms

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Functions for conversion to canonical forms

In [ ]:
def mps_orthonormalize_left(Alist):
    """
    Left-orthonormalize a MPS using QR decompositions.
    The list of tensors in `Alist` are updated in-place.

    Returns the overall norm of the original MPS. (The updated MPS has norm 1.)
    """
    # TODO: implement this function

In [ ]:
def mps_orthonormalize_right(Alist):
    """
    Right-orthonormalize a MPS using QR decompositions.
    The list of tensors in `Alist` are updated in-place.

    Returns the overall norm of the original MPS. (The updated MPS has norm 1.)
    """
    # TODO: implement this function

In [ ]:
def mps_orthonormalize_center(Alist, j):
    """
    Convert a MPS to site-canonical form with center at site `j`, such that
    all tensors to the left are left-orthonormal, and
    all tensors to the right are right-orthonormal.
    The list of tensors in `Alist` are updated in-place.
    """
    # TODO: implement this function

In [ ]:
def mps_orthonormalize_bond(Alist, j):
    """
    Convert a MPS to bond-canonical form, with a list of "singular values"
    between the `j`-th and `j+1`-th tensor.
    The list of tensors in `Alist` are updated in-place.

    Returns the singular value list.
    """
    # TODO: implement this function

## Utility functions

In [ ]:
def is_left_orthonormal(A):
    """
    Test whether a MPS tensor `A` is left-orthonormal.
    """
    s = A.shape
    assert len(s) == 3
    A = np.reshape(A, (s[0]*s[1], s[2]))
    return np.allclose(A.conj().T @ A, np.identity(s[2]))

In [ ]:
def is_right_orthonormal(A):
    """
    Test whether a MPS tensor `A` is right-orthonormal.
    """
    # call `is_left_orthonormal` with flipped left and right virtual bond dimensions
    return is_left_orthonormal(np.transpose(A, (0, 2, 1)))

In [ ]:
def mps_to_full_tensor(Alist):
    """
    Construct the full tensor corresponding to the MPS tensors `Alist`.

    The i-th MPS tensor Alist[i] is expected to have dimensions (n[i], D[i], D[i+1]),
    with `n` the list of logical dimensions and `D` the list of virtual bond dimensions.

    Note: Should only be used for debugging and testing.
    """
    # consistency check: dummy singleton dimension
    assert Alist[0].ndim == 3 and Alist[0].shape[1] == 1
    # formally remove dummy singleton dimension
    T = np.reshape(Alist[0], (Alist[0].shape[0], Alist[0].shape[2]))
    # contract virtual bonds
    for i in range(1, len(Alist)):
        T = np.tensordot(T, Alist[i], axes=(-1, 1))
    # consistency check: trailing dummy singleton dimension
    assert T.shape[-1] == 1
    # formally remove trailing singleton dimension
    T = np.reshape(T, T.shape[:-1])
    return T

In [ ]:
def mps_bond_to_full_tensor(Alist, S, j):
    """
    Construct the full tensor corresponding to the bond-canonical MPS
    with tensors `Alist` and "bond" singular values `S` between
    the `j`-th and `j+1`-th tensor.
    """
    # absorb bond singular values into j-th tensor
    Blist = [np.tensordot(Alist[i], np.diag(S), (2, 1)) if i==j else Alist[i] for i in range(len(Alist))]
    return mps_to_full_tensor(Blist)

In [ ]:
def partial_trace(rho, dimA, dimB):
    """
    Compute the partial traces of a density matrix 'rho' of a composite quantum system AB.

    Args:
        rho:  density matrix of dimension dimA*dimB x dimA*dimB
        dimA: dimension of subsystem A
        dimB: dimension of subsystem B
    Returns:
        tuple: reduced density matrices for subsystems A and B
    """
    # explicit subsystem dimensions
    rho = np.reshape(rho, (dimA, dimB, dimA, dimB))
    # trace out subsystem B
    rhoA = np.trace(rho, axis1=1, axis2=3)
    # trace out subsystem A
    rhoB = np.trace(rho, axis1=0, axis2=2)
    return rhoA, rhoB

In [ ]:
def crandn(size):
    """
    Draw random samples from the standard complex normal (Gaussian) distribution.
    """
    # 1/sqrt(2) is a normalization factor
    return (np.random.normal(size=size) + 1j*np.random.normal(size=size)) / np.sqrt(2)

In [ ]:
def xlogx(x):
    """
    Compute `x * log(x)` (pointwise), such that the result is zero for `x = 0`.
    """
    y = np.zeros_like(x)
    idx = x > 0
    y[idx] = x[idx] * np.log(x[idx])
    return y

## Examples and tests

In [ ]:
# logical and virtual bond dimensions (rather arbitrarily chosen)
n = [2, 5, 3, 4, 6, 3]
D = [1, 3, 4, 7, 6, 5, 1]

In [ ]:
# random MPS tensors (the scaling factor keeps the norm of the full tensor in a reasonable range)
np.random.seed(142)
Aref = [0.3 * crandn((n[i], D[i], D[i+1])) for i in range(len(n))]

# the tensors are randomly chosen, and in particular not of any normal form
print([is_left_orthonormal(A) for A in Aref])
print([is_right_orthonormal(A) for A in Aref])

# construct the full (dense) tensor which this MPS represents, as reference (should only be constructed for testing and debugging)
Tref = mps_to_full_tensor(Aref)
# its shape must be equal to `n` from above:
print("Tref.shape:", Tref.shape)

### Left-orthonormalization

In [ ]:
# first make a copy of the input tensors
AL = [A.copy() for A in Aref]

# function returns norm of input MPS
nrmL = mps_orthonormalize_left(AL)

In [ ]:
# these should all be True
[is_left_orthonormal(A) for A in AL]

In [ ]:
nrmL

In [ ]:
# compare norm with reference
abs(nrmL - np.linalg.norm(np.reshape(Tref, -1))) / abs(nrmL)

In [ ]:
# compare full tensor with reference: difference should be zero (up to numerical rounding errors)
np.linalg.norm(nrmL*mps_to_full_tensor(AL) - Tref)

### Right-orthonormalization

In [ ]:
# first make a copy of the input tensors
AR = [A.copy() for A in Aref]

# function returns norm of input MPS
nrmR = mps_orthonormalize_right(AR)

In [ ]:
# these should all be True
[is_right_orthonormal(A) for A in AR]

In [ ]:
nrmR

In [ ]:
# compare norm with reference
abs(nrmR - np.linalg.norm(np.reshape(Tref, -1))) / abs(nrmR)

In [ ]:
# compare full tensor with reference: difference should be zero (up to numerical rounding errors)
np.linalg.norm(nrmR*mps_to_full_tensor(AR) - Tref)

### Site-canonical form

In [ ]:
# again make a copy first
AC = [A.copy() for A in Aref]

# tensors are updated in-place, and overall norm is preserved (function has no formal return value)
jcenter = 2
mps_orthonormalize_center(AC, jcenter)

In [ ]:
# these should all be True
[is_left_orthonormal(A) for A in AC[:jcenter]]

In [ ]:
# these should all be True
[is_right_orthonormal(A) for A in AC[jcenter+1:]]

In [ ]:
# "center" tensor is not orthonormal in general
is_left_orthonormal(AC[jcenter]) or is_right_orthonormal(AC[jcenter])

In [ ]:
# compare full tensor with reference: difference should be zero (up to numerical rounding errors)
np.linalg.norm(mps_to_full_tensor(AC) - Tref)

### Bond-canonical form

In [ ]:
# again make a copy first
AB = [A.copy() for A in Aref]

jbond = 3
S = mps_orthonormalize_bond(AB, jbond)

In [ ]:
# list of singular values for "cut" at `jbond`
S

In [ ]:
# these should all be True
[is_left_orthonormal(AB[j]) if j <= jbond else is_right_orthonormal(AB[j]) for j in range(len(AB))]

In [ ]:
# compare full tensor with reference: difference should be zero (up to numerical rounding errors)
np.linalg.norm(mps_bond_to_full_tensor(AB, S, jbond) - Tref)

### Bond-singular values and entanglement entropy

In [ ]:
# compute (reduced) density matrices, as reference
ρref = np.outer(Tref, Tref.conj())
ρA, ρB = partial_trace(ρref, np.prod(n[:jbond+1]), np.prod(n[jbond+1:]))
print("ρA.shape:", ρA.shape)
print("ρB.shape:", ρB.shape)

In [ ]:
# must be Hermitian
np.linalg.norm(ρA - ρA.conj().T)

In [ ]:
# must be Hermitian
np.linalg.norm(ρB - ρB.conj().T)

In [ ]:
λA = np.linalg.eigvalsh(ρA)
λB = np.linalg.eigvalsh(ρB)

In [ ]:
# most of them are actually zero
λA

In [ ]:
# filter out zero eigenvalues
λA = λA[np.logical_not(np.isclose(λA, 0, atol=1e-13))]
λB = λB[np.logical_not(np.isclose(λB, 0, atol=1e-13))]

# sort in descending order
λA = np.sort(λA)[::-1]
λB = np.sort(λB)[::-1]

In [ ]:
λA

In [ ]:
# compare: should agree
np.linalg.norm(λA - λB)

In [ ]:
# compare: should agree with bond-singular values from above
np.linalg.norm(λA - S**2)

In [ ]:
# normalize singular values
Snrm = S / np.linalg.norm(S)
Snrm

In [ ]:
plt.semilogy(range(1, len(Snrm) + 1), Snrm**2, '.')
plt.ylabel("$\\sigma_j^2$")
plt.xlabel("$j$")
plt.title("normalized singular values for cut at bond {}".format(jbond))
plt.show()

In [ ]:
# finally compute entanglement entropy
np.sum(-xlogx(Snrm**2))